# Retrieve Dataset

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"stefanovski","key":"7cc1fc34fbd4cde54785510fb03b40fe"}'}

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c 'store-sales-time-series-forecasting'

store-sales-time-series-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
! unzip store-sales-time-series-forecasting.zip -d data

Archive:  store-sales-time-series-forecasting.zip
replace data/holidays_events.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/holidays_events.csv  
  inflating: data/oil.csv            
  inflating: data/sample_submission.csv  
  inflating: data/stores.csv         
  inflating: data/test.csv           
  inflating: data/train.csv          
  inflating: data/transactions.csv   


# Read Data

In [ ]:
import numpy as np
import os
import pandas as pd
import datetime
import torch
import torch.nn as nn
import tensorflow as tf
from tensorflow import keras
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from keras import backend
from keras.utils import losses_utils
from keras.losses import LossFunctionWrapper
from keras.metrics import MeanMetricWrapper
from keras.dtensor import utils as dtensor_utils

In [ ]:
train = pd.read_csv('data/train.csv', parse_dates=["date"])
test = pd.read_csv('data/test.csv', parse_dates=["date"])
stores = pd.read_csv('data/stores.csv')
oil = pd.read_csv('data/oil.csv', parse_dates=["date"])
holidays_events = pd.read_csv('data/holidays_events.csv', parse_dates=["date"])
transactions = pd.read_csv('data/transactions.csv', parse_dates=["date"])

In [ ]:
train['family'] = pd.Categorical(train['family'])
test['family'] = pd.Categorical(test['family'])
stores['city'] = pd.Categorical(stores['city'])
stores['state'] = pd.Categorical(stores['state'])
stores['type'] = pd.Categorical(stores['type'])
holidays_events['type'] = pd.Categorical(holidays_events['type'])
holidays_events['locale'] = pd.Categorical(holidays_events['locale'])
holidays_events['locale_name'] = pd.Categorical(holidays_events['locale_name'])

In [ ]:
split_set = train.pivot(index=train[['date', 'store_nbr']], columns='family', values=['sales', 'onpromotion']).unstack()

In [ ]:
# Flatten columns from 3 -> 1 layers
new_store_nbr = split_set.columns.get_level_values('store_nbr').astype('str').unique()
split_set.columns.set_levels(
    [new_store_nbr],
    level=['store_nbr'],
    inplace=True
)
split_set.columns = ['_'.join(col).strip() for col in split_set.columns.values]

<ipython-input-349-38cf1e3f939b>:3: FutureWarning: inplace is deprecated and will be removed in a future version.
  split_set.columns.set_levels(


In [ ]:
split_set = split_set.dropna(axis='columns')

In [ ]:
split_set.isna().sum().sum()

0

## Split

In [ ]:
split = {
    'train':      0.70,
    'validation': 0.20,
    'test':       0.10
}
dates = split_set.index.unique()
date_length = len(dates)
first_day = dates[0]
# first_day = dates[365]  # Skip the first year
date_length = len(dates)
train_end = dates[int(date_length*split['train'])]
val_end = dates[int(date_length*(split['train']+split['validation']))]
last_day = dates[-1]

In [ ]:
train_df = split_set[first_day:train_end]
val_df = split_set[train_end:val_end]
test_df = split_set[val_end:last_day]

In [ ]:
# Normalize data to prepare for tf. This should be changed to moving avg.
train_mean = train_df.mean()
train_std = train_df.std()
zero_std = train_std.loc[train_std == 0]
no_nan_mean = train_mean.drop(zero_std.index)
no_nan_std = train_std.drop(zero_std.index)

train_df = ((train_df.drop(zero_std.index, axis='columns') - no_nan_mean) / no_nan_std)
val_df = ((val_df.drop(zero_std.index, axis='columns') - no_nan_mean) / no_nan_std)
test_df = ((test_df.drop(zero_std.index, axis='columns') - no_nan_mean) / no_nan_std)
# kaggle_df = (encoded_kaggle - train_mean) / train_std # We do not have access to test avg.

## Windowing

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
          axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

  def plot(self, model=None, plot_col='sales', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
      plt.subplot(max_n, 1, n+1)
      plt.ylabel(f'{plot_col} [normed]')
      plt.plot(self.input_indices, inputs[n, :, plot_col_index],
              label='Inputs', marker='.', zorder=-10)

      if self.label_columns:
        label_col_index = self.label_columns_indices.get(plot_col, None)
      else:
        label_col_index = plot_col_index

      if label_col_index is None:
        continue

      plt.scatter(self.label_indices, labels[n, :, label_col_index],
                  edgecolors='k', label='Labels', c='#2ca02c', s=64)
      if model is not None:
        predictions = model(inputs)
        plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                    marker='X', edgecolors='k', label='Predictions',
                    c='#ff7f0e', s=64)

      if n == 0:
        plt.legend()

    plt.xlabel('date')

  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result


# Construct Model

In [ ]:
def root_mean_squared_logarithmic_error(y_true, y_pred):
  y_pred = tf.convert_to_tensor(y_pred)
  y_true = tf.cast(y_true, y_pred.dtype)
  first_log = tf.math.log(backend.maximum(y_pred, backend.epsilon()) + 1.0)
  second_log = tf.math.log(backend.maximum(y_true, backend.epsilon()) + 1.0)
  return tf.math.sqrt(backend.mean(
      tf.math.squared_difference(first_log, second_log), axis=-1
  ))

In [ ]:
class LOSS_RootMeanSquaredLogarithmicError(LossFunctionWrapper):
    def __init__(
        self,
        reduction=losses_utils.ReductionV2.AUTO,
        name="root_mean_squared_logarithmic_error",
    ):
        super().__init__(
            root_mean_squared_logarithmic_error, name=name, reduction=reduction
        )

class METRIC_RootMeanSquaredLogarithmicError(MeanMetricWrapper):
    @dtensor_utils.inject_mesh
    def __init__(self, name="root_mean_squared_logarithmic_error", dtype=None):
        super().__init__(root_mean_squared_logarithmic_error, name, dtype=dtype)


In [ ]:
# loss = tf.keras.losses.MeanSquaredLogarithmicError()
# metric = tf.keras.metrics.MeanSquaredLogarithmicError()
loss = LOSS_RootMeanSquaredLogarithmicError
metric = METRIC_RootMeanSquaredLogarithmicError
optimizer = tf.keras.optimizers.Adam
MAX_EPOCHS = 4

In [ ]:
def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=loss(),
                optimizer='adam',
                metrics=[metric()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

# LSTM, Multi-Step

In [ ]:
targets = [col for col in train_df.columns if 'sales' in col]

In [ ]:
OUT_STEPS = 16
multi_window = WindowGenerator(input_width=20,
                               label_columns=targets,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

In [ ]:
num_features = len(targets)

In [ ]:
def const_multi_lstm_model():
  # strategy = tf.distribute.TPUStrategy(resolver)
  # with strategy.scope():
  model = tf.keras.Sequential([
              # Shape [batch, time, features] => [batch, lstm_units].
              # Adding more `lstm_units` just overfits more quickly.
              tf.keras.layers.LSTM(32, return_sequences=False),
              # Shape => [batch, out_steps*features].
              tf.keras.layers.Dense(OUT_STEPS*num_features,
                                    kernel_initializer=tf.initializers.zeros()),
              # Shape => [batch, out_steps, features].
              tf.keras.layers.Reshape([OUT_STEPS, num_features])
          ])
  return model

In [ ]:
# Use GPU acceleration for fastest results
model = const_multi_lstm_model()

history = compile_and_fit(model, multi_window)

# IPython.display.clear_output()

# multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
# multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)

Epoch 1/4
36/36 [==============================] - 8s 171ms/step - loss: 0.3923 - root_mean_squared_logarithmic_error: 0.3923 - val_loss: 0.6610 - val_root_mean_squared_logarithmic_error: 0.6610
Epoch 2/4
36/36 [==============================] - 5s 139ms/step - loss: 0.3923 - root_mean_squared_logarithmic_error: 0.3923 - val_loss: 0.6610 - val_root_mean_squared_logarithmic_error: 0.6610
Epoch 3/4
36/36 [==============================] - 5s 139ms/step - loss: 0.3923 - root_mean_squared_logarithmic_error: 0.3923 - val_loss: 0.6610 - val_root_mean_squared_logarithmic_error: 0.6610


In [ ]:
results = model.predict(multi_window.test)

5/5 [==============================] - 5s 10ms/step


In [ ]:
results.shape

(134, 16, 1651)

In [ ]:
averaged = np.mean(results, axis=0)
averaged.shape

(16, 1651)

In [ ]:
result_df = pd.DataFrame(
    averaged,
    index = test['date'].unique(),
    columns = targets
)

In [ ]:
rescaled = ((result_df * no_nan_std) + no_nan_mean).dropna(axis='columns') # Drop na since onpromotion introduces it

In [ ]:
tuples = [x.split('_')[::] for x in rescaled.columns]

In [ ]:
midx = pd.MultiIndex.from_tuples(tuples)
midx

MultiIndex([('sales', 'AUTOMOTIVE',  '1'),
            ('sales', 'AUTOMOTIVE', '10'),
            ('sales', 'AUTOMOTIVE', '11'),
            ('sales', 'AUTOMOTIVE', '12'),
            ('sales', 'AUTOMOTIVE', '13'),
            ('sales', 'AUTOMOTIVE', '14'),
            ('sales', 'AUTOMOTIVE', '15'),
            ('sales', 'AUTOMOTIVE', '16'),
            ('sales', 'AUTOMOTIVE', '17'),
            ('sales', 'AUTOMOTIVE', '18'),
            ...
            ('sales',    'SEAFOOD', '49'),
            ('sales',    'SEAFOOD',  '5'),
            ('sales',    'SEAFOOD', '50'),
            ('sales',    'SEAFOOD', '51'),
            ('sales',    'SEAFOOD', '53'),
            ('sales',    'SEAFOOD', '54'),
            ('sales',    'SEAFOOD',  '6'),
            ('sales',    'SEAFOOD',  '7'),
            ('sales',    'SEAFOOD',  '8'),
            ('sales',    'SEAFOOD',  '9')],
           length=1651)

In [ ]:
rescaled.columns = midx

In [ ]:
stacked = rescaled.stack().stack()
stacked

sales
2017-08-16 1 AUTOMOTIVE                     2.870229
             BEAUTY                         2.077184
             BEVERAGES                   1362.569975
             BREAD/BAKERY                 320.048038
             CELEBRATION                    7.594572
...                                              ...
2017-08-31 9 POULTRY                      493.628354
             PREPARED FOODS                77.586248
             PRODUCE                      677.812849
             SCHOOL AND OFFICE SUPPLIES     3.572519
             SEAFOOD                       16.562319

[26416 rows x 1 columns]

In [ ]:
dummy = test.copy()
dummy['dummy_sales'] = 0

In [ ]:
new = dummy.drop(['onpromotion', 'id'], axis=1).set_index(['date', 'store_nbr', 'family'])
new

dummy_sales
date       store_nbr family                                 
2017-08-16 1         AUTOMOTIVE                            0
                     BABY CARE                             0
                     BEAUTY                                0
                     BEVERAGES                             0
                     BOOKS                                 0
...                                                      ...
2017-08-31 9         POULTRY                               0
                     PREPARED FOODS                        0
                     PRODUCE                               0
                     SCHOOL AND OFFICE SUPPLIES            0
                     SEAFOOD                               0

[28512 rows x 1 columns]

In [ ]:
stacked.index.set_names('date', level=0, inplace=True)
stacked.index.set_names('store_nbr', level=1, inplace=True)
stacked.index.set_names('family', level=2, inplace=True)

In [ ]:
new.index

MultiIndex([('2017-08-16', 1,                 'AUTOMOTIVE'),
            ('2017-08-16', 1,                  'BABY CARE'),
            ('2017-08-16', 1,                     'BEAUTY'),
            ('2017-08-16', 1,                  'BEVERAGES'),
            ('2017-08-16', 1,                      'BOOKS'),
            ('2017-08-16', 1,               'BREAD/BAKERY'),
            ('2017-08-16', 1,                'CELEBRATION'),
            ('2017-08-16', 1,                   'CLEANING'),
            ('2017-08-16', 1,                      'DAIRY'),
            ('2017-08-16', 1,                       'DELI'),
            ...
            ('2017-08-31', 9,                  'MAGAZINES'),
            ('2017-08-31', 9,                      'MEATS'),
            ('2017-08-31', 9,              'PERSONAL CARE'),
            ('2017-08-31', 9,               'PET SUPPLIES'),
            ('2017-08-31', 9,    'PLAYERS AND ELECTRONICS'),
            ('2017-08-31', 9,                    'POULTRY'),
        

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
       '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
       '50', '51', '52', '53', '54'],
      dtype='object', name='store_nbr')

In [ ]:
new_store_nbr_idx = new.index.get_level_values('store_nbr').unique().sort_values().astype('str')
new.index.set_levels(
    [new_store_nbr_idx],
    level=['store_nbr'],
    inplace=True
)

<ipython-input-467-d6c497d0b5ea>:2: FutureWarning: inplace is deprecated and will be removed in a future version.
  new.index.set_levels(


In [ ]:
new.index

MultiIndex([('2017-08-16', '1',                 'AUTOMOTIVE'),
            ('2017-08-16', '1',                  'BABY CARE'),
            ('2017-08-16', '1',                     'BEAUTY'),
            ('2017-08-16', '1',                  'BEVERAGES'),
            ('2017-08-16', '1',                      'BOOKS'),
            ('2017-08-16', '1',               'BREAD/BAKERY'),
            ('2017-08-16', '1',                'CELEBRATION'),
            ('2017-08-16', '1',                   'CLEANING'),
            ('2017-08-16', '1',                      'DAIRY'),
            ('2017-08-16', '1',                       'DELI'),
            ...
            ('2017-08-31', '9',                  'MAGAZINES'),
            ('2017-08-31', '9',                      'MEATS'),
            ('2017-08-31', '9',              'PERSONAL CARE'),
            ('2017-08-31', '9',               'PET SUPPLIES'),
            ('2017-08-31', '9',    'PLAYERS AND ELECTRONICS'),
            ('2017-08-31', '9',        

In [ ]:
stacked.index

MultiIndex([('2017-08-16', '1',                 'AUTOMOTIVE'),
            ('2017-08-16', '1',                     'BEAUTY'),
            ('2017-08-16', '1',                  'BEVERAGES'),
            ('2017-08-16', '1',               'BREAD/BAKERY'),
            ('2017-08-16', '1',                'CELEBRATION'),
            ('2017-08-16', '1',                   'CLEANING'),
            ('2017-08-16', '1',                      'DAIRY'),
            ('2017-08-16', '1',                       'DELI'),
            ('2017-08-16', '1',                       'EGGS'),
            ('2017-08-16', '1',               'FROZEN FOODS'),
            ...
            ('2017-08-31', '9',                  'MAGAZINES'),
            ('2017-08-31', '9',                      'MEATS'),
            ('2017-08-31', '9',              'PERSONAL CARE'),
            ('2017-08-31', '9',               'PET SUPPLIES'),
            ('2017-08-31', '9',    'PLAYERS AND ELECTRONICS'),
            ('2017-08-31', '9',        

In [ ]:
joined = new.join(stacked, how='outer')

In [ ]:
joined = joined.drop('dummy_sales', axis='columns')

In [ ]:
joined = joined.fillna(0)

In [ ]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [ ]:
joined = joined.reset_index()

In [ ]:
joined

,date,store_nbr,family,sales
0,2017-08-16,1,AUTOMOTIVE,2.870229
1,2017-08-16,1,BABY CARE,0.000000
2,2017-08-16,1,BEAUTY,2.077184
3,2017-08-16,1,BEVERAGES,1362.569975
4,2017-08-16,1,BOOKS,0.000000
...,...,...,...,...
28507,2017-08-31,9,POULTRY,493.628354
28508,2017-08-31,9,PREPARED FOODS,77.586248
28509,2017-08-31,9,PRODUCE,677.812849
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,3.572519


In [ ]:
final_submission = joined.join(test, lsuffix='_l')[['id', 'sales']]
final_submission

,id,sales
0,3000888,2.870229
1,3000889,0.000000
2,3000890,2.077184
3,3000891,1362.569975
4,3000892,0.000000
...,...,...
28507,3029395,493.628354
28508,3029396,77.586248
28509,3029397,677.812849
28510,3029398,3.572519


In [ ]:
final_submission.to_csv('submission.csv', index=False)